In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import json

# Load synthetic images from batch_images/1-100.jpg and labels from JSON
synthetic_images = []
loaded_labels = []

for i in range(1, 101):
    image_path = f"final_synth_data/{i}.jpg"
    label_index = i - 1  # Index for labels starting from 0

    if os.path.exists(image_path):
        img = Image.open(image_path)
        img = img.convert("L")  # Convert to grayscale if not already
        img = img.resize((1024, 1024))  # Resize to a uniform size (adjust dimensions as needed)
        img_array = np.array(img)
        
        synthetic_images.append(img_array)
        
        with open('final_synth_data/done_labels.json') as json_file:
            labels_data = json.load(json_file)
            labels = labels_data['labels']

            # Check if the label for this index exists
            if label_index < len(labels):
                label = labels[label_index]['label']

                age = int(label[0])
                gender = 'Muž' if label[1] == 0.0 else 'Žena'
                cardiovascular_disease = 'Ano' if label[3] == 1.0 else 'Ne'
                smoker = 'Ano' if label[4] == 1.0 else 'Ne'
                sport_hobby_with_fingers = 'Ano' if label[5] == 1.0 else 'Ne'
                alcohol_before_scan = 'Ano' if label[6] == 1.0 else 'Ne'
                skin_disease = 'Ano' if label[7] == 1.0 else 'Ne'

                info_str = f"Věk: {age}\nPohlaví: {gender}\nKardiovaskulární choroba: {cardiovascular_disease}\nKouření: {smoker}\nSport/Hobby s prsty: {sport_hobby_with_fingers}\nAlkohol před skenem: {alcohol_before_scan}\nKožní onemocnění: {skin_disease}"

                loaded_labels.append(info_str)
            else:
                print(f"Štítek pro obrázek {i} nebyl nalezen v JSON souboru.")
    else:
        pass  # Ignore missing images

synthetic_images = np.array(synthetic_images)

unique_labels_set = set()


missing_labels = []
while len(missing_labels) < 16:
    new_label = np.random.randint(0, 2, size=5)  
    new_label_str = ','.join(str(val) for val in new_label)
    if new_label_str not in unique_labels_set:
        unique_labels_set.add(new_label_str)
        unique_ages = list(set([int(label.split('\n')[0].split(': ')[1]) for label in loaded_labels]))
        random_age = np.random.choice(unique_ages)
        gender = 'Muž' if np.random.randint(0, 2) == 0 else 'Žena'  # Random gender

        # Find label info for the selected age
        for label_info in loaded_labels:
            if f"Věk: {random_age}" in label_info:
                info_str = label_info
                break

        # Replace label values except age and gender
        label_values = new_label_str.split(',')
        info_str = f"Věk: {random_age}\nPohlaví: {gender}\nKardiovaskulární choroba: {'Ano' if label_values[0] == '1' else 'Ne'}\nKouření: {'Ano' if label_values[1] == '1' else 'Ne'}\nSport/Hobby s prsty: {'Ano' if label_values[2] == '1' else 'Ne'}\nAlkohol před skenem: {'Ano' if label_values[3] == '1' else 'Ne'}\nKožní onemocnění: {'Ano' if label_values[4] == '1' else 'Ne'}"

        missing_labels.append(info_str)

# Display the synthetic images in a 3x3 matrix with their corresponding label information
fig, axs = plt.subplots(4, 4, figsize=(16, 16), dpi=400)  # Increase figure size and DPI for higher resolution

for i, info_str in enumerate(missing_labels):
    # Resize the image for display
    resized_img = Image.fromarray(synthetic_images[i]).resize((2048, 2048))
    
    # Display the resized image
    axs[i // 4, i % 4].imshow(np.array(resized_img), cmap='gray', interpolation='bilinear')  # Using bilinear interpolation for sharper images
    axs[i // 4, i % 4].axis('off')

    # Annotate label information in the corners
    axs[i // 4, i % 4].text(5, 5, info_str, fontsize=8, color='white', backgroundcolor='black', verticalalignment='top')

plt.tight_layout(pad=2)
plt.show()